In [ ]:
import scanpy as sc
import besca as bc
import os
import bbknn
import numpy as np
import scvelo as scv
import pandas as pd
import cellrank as cr

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from scipy import stats
import itertools


In [ ]:
color_dict = {'R': 'coral', 'TF': 'firebrick', 'NR_nadj': 'lightskyblue','NR_adj': 'royalblue'}


In [ ]:
sc.logging.print_versions()

In [ ]:
species='human'
scv.settings.set_figure_params('scvelo')

# Reading the h5ad

In [ ]:
WD='/Fullanalysis/'
analysis_name='sw_besca2_immune'
dcdata=sc.read('/Fullanalysis/analyzed/sw_besca2_immune/sw_besca2_immune.annotated.filtered.h5ad')

In [ ]:
outpath='Fullanalysis/analyzed/sw_besca2_immune/velocity/'

In [ ]:
velosubset='MonoMacro'

In [ ]:
set(dcdata.obs['celltype2_pub'])

In [ ]:
sc.set_figure_params()

In [ ]:
sc.pl.umap(dcdata, color=['leiden', 'leiden_TNK', 'Sample type'],legend_loc='on data', legend_fontsize=0.6)

In [ ]:
sc.set_figure_params()
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_rc_cells'], size=8)

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_merged'], size=8)

In [ ]:
sc.set_figure_params()
sc.pl.umap(dcdata,color=['CD8A','CD8B','CD4','CD3D'], color_map='viridis')

In [ ]:
set(dcdata.obs['celltype3_pub'])

In [ ]:
set(dcdata.obs['celltype2_pub'])

In [ ]:
dcdata=dcdata[(dcdata.obs['celltype2_pub'].isin(['macrophage','classical monocyte','non-classical monocyte']))].copy()


In [ ]:
dcdata.obs['leidenf']=dcdata.obs['leiden'].copy()

In [ ]:
sc.pl.umap(dcdata, color=['leiden', 'celltype3_pub','celltype3', 'Sample type'])

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_pub'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['CD8A','CD8B','CD4','CD3D'])

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_rc_cells'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
#sc.pp.neighbors(dcdata)
bbknn.bbknn(dcdata, batch_key='PatientID', use_rep='X_pca')

In [ ]:
sc.tl.diffmap(dcdata)


sc.pp.neighbors(dcdata, n_neighbors=10, use_rep='X_diffmap')
sc.tl.leiden(dcdata, resolution=0.75)


sc.tl.paga(dcdata, groups='leiden')

sc.pl.umap(dcdata,color='leiden')


In [ ]:
sc.pl.paga(dcdata)


In [ ]:

sc.tl.umap(dcdata, init_pos='paga')

sc.pl.umap(dcdata,color=['leiden','leidenf'],legend_loc='on data')

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_pub'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['PatientID'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['Sample type'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
if skipread==True:
    loom=scv.read('/pstore/data/biomics/_pre_portfolio/ONC/scSEQ/ROCHE_INHOUSE/Schlieren_Project_Support_8/Fullanalysis/loom_afterconcat.h5ad')

In [ ]:
adata=scv.utils.merge(dcdata, loom).copy()

In [ ]:
#adata.write("/pstore/data/biomics/_pre_portfolio/ONC/scSEQ/ROCHE_INHOUSE/Schlieren_Project_Support_8/Fullanalysis/CD8Tcells-loom.h5ad", compression="gzip")


In [ ]:
#adata=scv.read("/pstore/data/biomics/_pre_portfolio/ONC/scSEQ/ROCHE_INHOUSE/Schlieren_Project_Support_8/Fullanalysis/CD8Tcells-loom.h5ad")

In [ ]:
sc.set_figure_params(6)
sc.pl.umap(adata,color=['PatientID'],ncols=2,
           save='-subsettrajectorymap-'+velosubset+'.PatientID.pdf')

In [ ]:
sc.pl.umap(adata,color=['RCat'],ncols=2,
           save='-subsettrajectorymap-'+velosubset+'.RCat.pdf')

In [ ]:
sc.pl.umap(adata,color=['Sample type'],ncols=2,
           save='-subsettrajectorymap-'+velosubset+'.SampleType.pdf')

In [ ]:
sc.pl.umap(adata,color=['Lesion'],ncols=2,
           save='-subsettrajectorymap-'+velosubset+'.Lesion.pdf')

In [ ]:
sc.set_figure_params(6)

In [ ]:
sc.pl.umap(adata,color=['leiden'],ncols=2,
           save='-subsettrajectorymap-'+velosubset+'.leiden.pdf', legend_loc='on data')

In [ ]:

#adata.write(WD+'/analyzed/'+analysis_name+'/'+analysis_name+'.annotated.'+velosubset+'.h5ad')


In [ ]:
adata=sc.read(WD+'/analyzed/'+analysis_name+'/'+analysis_name+'.annotated.'+velosubset+'.h5ad')

In [ ]:
##### Take monocytes only partially in #####

subdata=adata[adata.obs['leiden'].isin(['5','29','12','14','16','19','22','23','7','13','7'])].copy() #'1',

In [ ]:
#sc.pp.neighbors(dcdata)
bbknn.bbknn(subdata, batch_key='PatientID', use_rep='X_pca')

sc.tl.diffmap(subdata)


sc.pp.neighbors(subdata, n_neighbors=10, use_rep='X_diffmap')
sc.tl.leiden(subdata, resolution=0.5)


sc.tl.paga(subdata, groups='leiden')

sc.pl.umap(subdata,color='leiden')


In [ ]:
#subdata
sc.pl.umap(adata,color=['leiden'],legend_loc='on data')

In [ ]:
#subdata
sc.pl.umap(subdata,color=['leiden'],legend_loc='on data')

In [ ]:
sc.pl.umap(adata,color=['leiden'],legend_loc='on data')

In [ ]:
catorder=corder=['2','5','10','7','13','8']

In [ ]:
sc.pl.umap(adata,color=['celltype3_pub'],
           save='-subsettrajectorymap-'+velosubset+'.celltype3_pub.pdf')

In [ ]:
scv.pp.filter_and_normalize(subdata)
scv.pp.moments(subdata)
scv.pp.neighbors(subdata)

scv.tl.recover_dynamics(subdata)
scv.tl.velocity(subdata,mode='dynamical')
scv.tl.velocity_graph(subdata)

In [ ]:
scv.pp.filter_and_normalize(adata)
scv.pp.moments(adata)
scv.pp.neighbors(adata)

scv.tl.recover_dynamics(adata)
scv.tl.velocity(adata,mode='dynamical')
scv.tl.velocity_graph(adata)

In [ ]:
mysubset='All'
subcat='PBMCandTIL'


In [ ]:
#subcat='TILonly'
subdata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.subsampled.h5ad')

In [ ]:
#subcat='TILonly'
adata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')

In [ ]:
adata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')
subdata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.subsampled.origmap.h5ad')

In [ ]:
sc.pl.umap(adata, color='RCat')

### Heatmaps 

In [ ]:
goi=list(set(['S100A12', 'S100A9', 'S100A8', 'LGALS2', 'LYZ', 'FCN1', 'C1QB', 'C1QA', 'C1QC', 'TREM2', 'LGMN', 
     'HLA-DQA1', 'HLA-DPA1', 'IL1B', 'PTX3', 
     'EREG', 'IL8', 'PTGS2', 'AREG', 'APOE', 'SLC7A8', 'A2M', 'TOP2A', 'CCL20', 
     'CXCL3', 'SPP1', 'HILPDA', 'SLAMF9', 'TNFSF14', 'SLC2A1', 'HK2', 
     'CXCL11', 'CXCL10', 'CXCL9', 'ISG15', 'IFIT3', 'MARCO', 'FN1', 'FBP1', 'MRC1', 
     'CLEC9A', 'C3', 'AXL', 'PLAC8', 'MARCKSL1', 'CD36', 'CD68', 
     'CCL4', 'EGR1', 'NR4A1', 'NR4A2', 'NLRP3', 'FOLR2', 'PTX3', 'G0S2', 'CD300E', 
     'GAS6', 'IL6', 'INHBA', 'GPNMB', 'IDO1', 'IFIT1', 'IFIT2',
     'CD274', 'VSIG4', 'EGR1', 'EGR2', 'EGR3', 'EGR4', 'NLRP3', 'TLR10', 'TLR1', 'TLR3', 'TLR7', 
     'CXCL12', 'TNF', 'IL18', 'NFKB1', 'SELL']))

In [ ]:
sc.pl.dotplot(subdata, var_names=goi,groupby='celltype3_pub')

In [ ]:
average_obs,fraction_obs=bc.get_means(subdata, 'leiden')



In [ ]:
subdata.uns['celltype3_pub_colors']


In [ ]:
sc.set_figure_params(6)

In [ ]:
sc.pl.umap(subdata, color=['leiden'], legend_loc='on data', save='-leiden-'+velosubset+'-'+mysubset+'-'+subcat+'.pdf')

In [ ]:
sc.pl.umap(subdata, color=['celltype3_pub'], save='-celltype3_pub-'+velosubset+'-'+mysubset+'-'+subcat+'.pdf')

In [ ]:
sc.pl.umap(subdata, color=['CXCL9','PTX3'], color_map='viridis', save='-CXCL9-PTX3-'+velosubset+'-'+mysubset+'-'+subcat+'.pdf')

In [ ]:
#'CCL3','CCL4', 'TIMP1', 'MIF', 'LGALS1','TREM1', 'MT1X'
sc.pl.umap(subdata, color=['TREM1','NR4A2'], color_map='viridis', save='-TREM1-NR4A2-'+velosubset+'-'+mysubset+'-'+subcat+'.pdf')

In [ ]:
goi=['S100A12',  'S100A8','FCN1', 'SELL', 'CD300E', 'MARCKSL1', 'CD68', 'CD163', 'MSR1',
'NR4A1','TREM2','HLA-DQA1', 'HLA-DPA1', 'NLRP3', 'FN1', 
'AREG', 'ISG15', 'IFIT1', 'IFIT3', 'IFI27','CXCL11', 'CXCL10', 'CXCL9','CD274','IDO1', 'TNF',
 'AXL',  'MARCO', 'PTX3', 'CD36', 'MRC1', 'GPNMB', 
'LGALS2','APOE', 'EREG', 'INHBA', 'HILPDA', 'SPP1',
'CXCL12',  'CCL20', 'CXCL3', 'IL8', 'CCL4', 'IL6', 'IL1B', 'IL18','FOLR2',
'C1QB', 'LGMN', 'GAS6','G0S2', 'C3',
      'SLAMF9', 'TNFSF14', 'VSIG4', 'TLR10', 'TLR1', 
     'TLR3', 'TLR7',  'EGR3', 'EGR4', 'NFKB1','SLC2A1', 'HK2', 'FBP1', 'CX3CR1','PTGS2']
#PTX3, IL8, EREG, PTGS2

In [ ]:

mycol=['#B79762','#0000A6','#B79762','#0000A6','#7A4900','#7A4900','#A30059','#3B5DFF','#0000A6','#B79762']

sns.set(font_scale=0.6)
#a=sns.clustermap(fraction_obs.loc[:,goi],col_cluster=True, row_cluster=True,figsize=(15,4),
#                 cmap='viridis',metric='correlation',row_colors=mycol, vmax=0.8, standard_scale=1) #row_colors=mycol,
a=sns.clustermap(average_obs.loc[:,goi],col_cluster=True, row_cluster=True,figsize=(15,4),
                 cmap='viridis',metric='correlation',row_colors=mycol, vmax=0.8, standard_scale=1) #row_colors=mycol,
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.dyn-subsampled.08.pdf')
#a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.dyn-subsampled.08.eps', format='eps')


In [ ]:
figdir

In [ ]:
sns.set(font_scale=0.6)
#a=sns.clustermap(fraction_obs.loc[:,goi],col_cluster=True, row_cluster=True,figsize=(15,4),
#                 cmap='viridis',metric='correlation',row_colors=mycol) #row_colors=mycol,
a=sns.clustermap(average_obs.loc[:,goi],col_cluster=True, row_cluster=True,figsize=(15,4),
                 cmap='viridis',metric='correlation',row_colors=mycol) #row_colors=mycol,
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.dyn-subsampled.pdf')
#a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.dyn-subsampled.eps', format='eps')

In [ ]:
DEgenes12vs19=bc.tl.dge.get_de(subdata[subdata.obs['leiden'].isin(['12','19'])],'leiden',demethod='wilcoxon',topnr=5000, logfc=1,padj=0.05)


In [ ]:
#DEgenes['0']
for i in DEgenes12vs19.keys():
    DEgenes12vs19[i].sort_values('Log2FC',ascending=False).to_csv(figdir+'/DEclustergenes_12vs19_'+velosubset+'-'+mysubset+'_'+subcat+'_subdata_leiden_'+i+'.tsv',sep='\t')

In [ ]:
DEgenes12vs19['12'].iloc[0:20,:]

In [ ]:
DEgenes12vs19['19'].iloc[0:30,:]

In [ ]:
DEgenes=bc.tl.dge.get_de(subdata,'leiden',demethod='wilcoxon',topnr=5000, logfc=1,padj=0.05)


In [ ]:
#DEgenes['0']
for i in DEgenes.keys():
    DEgenes[i].sort_values('Log2FC',ascending=False).to_csv(figdir+'/Topclustergenes_'+velosubset+'-'+mysubset+'_'+subcat+'_subdata_leiden_'+i+'.tsv',sep='\t')

In [ ]:
clustercomp=bc.tl.count_occurrence_subset(adata, 'leiden', count_variable ='PatientID', return_percentage = True)
clustercomp[clustercomp.max().sort_values().index].transpose()

sc.set_figure_params()
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,5)

clustercomp=clustercomp[clustercomp.max().sort_values().index].copy()
ax=clustercomp.transpose().plot(kind='bar', stacked=True)
ax.set_ylabel('Percentage donor per cluster')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.figure.savefig(figdir+'Clustercomp-'+velosubset+'.PatientID.pdf',bbox_inches='tight')
ax.figure.savefig(figdir+'Clustercomp-'+velosubset+'.PatientID.eps', format='eps')

In [ ]:
clustercomp=bc.tl.count_occurrence_subset(adata, 'leiden', count_variable ='RCat', return_percentage = True)
clustercomp[clustercomp.max().sort_values().index].transpose()

sc.set_figure_params()
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,5)

clustercomp=clustercomp[clustercomp.max().sort_values().index].copy()
ax=clustercomp.transpose().plot(kind='bar', stacked=True)
ax.set_ylabel('Percentage donor per cluster')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.figure.savefig(figdir+'Clustercomp-'+velosubset+'.RCat.pdf',bbox_inches='tight')
ax.figure.savefig(figdir+'Clustercomp-'+velosubset+'.RCat.eps', format='eps')

In [ ]:
clustercomp=bc.tl.count_occurrence_subset(adata, 'leiden', count_variable ='Lesion', return_percentage = True)
clustercomp[clustercomp.max().sort_values().index].transpose()

sc.set_figure_params()
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,5)

clustercomp=clustercomp[clustercomp.max().sort_values().index].copy()
ax=clustercomp.transpose().plot(kind='bar', stacked=True)
ax.set_ylabel('Percentage donor per cluster')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.figure.savefig(figdir+'Clustercomp-'+velosubset+'.Lesion.pdf',bbox_inches='tight')
ax.figure.savefig(figdir+'Clustercomp-'+velosubset+'.Lesion.eps', format='eps')

In [ ]:
sc.set_figure_params()

In [ ]:
scv.tl.velocity_confidence(subdata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(subdata, c=keys, cmap='coolwarm', perc=[5, 95])


In [ ]:
scv.tl.velocity_confidence(adata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c=keys, cmap='coolwarm', perc=[5, 95])


In [ ]:
scv.pl.velocity_graph(adata, threshold=.2, color='leiden')

In [ ]:
scv.pl.velocity_graph(subdata, threshold=.2, color='leiden')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden','Sample type'], save='-Velocity')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden','Sample type'],
                                 save=scv.settings.figdir+'-trajectorymap-EntireMap-'+velosubset+'-'+mysubset+'-'+subcat+'.Velocity-Sample_Type.svg')

In [ ]:
sc.set_figure_params(10)

In [ ]:
scv.settings.plot_prefix = ""

In [ ]:
scv.settings.figdir='sandbox/'

In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['leiden','Sample type'],
                                save=scv.settings.figdir+'-trajectorymap-'+velosubset+'-'+mysubset+'-'+subcat+'.Velocity-Sample_Type.svg')

In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['leiden','Sample type'],
                                save=scv.settings.figdir+'-trajectorymap-'+velosubset+'-'+mysubset+'-'+subcat+'.Velocity-Sample_Type.pdf')

In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['leiden','Sample type'],
                                save=scv.settings.figdir+'-trajectorymap-'+velosubset+'-'+mysubset+'-'+subcat+'.Velocity-Sample_Type.png')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['celltype3_pub'])

In [ ]:
#sc.set_figure_params(10)
#import matplotlib.pyplot as plt
#plt.rcParams["figure.figsize"] = (12,10)

In [ ]:
figdir=outpath


In [ ]:
figdir=os.path.join(outpath+'/publication/')
scv.settings.figdir = figdir
if not os.path.exists(figdir):
    os.makedirs(figdir)

In [ ]:
scv.tl.score_genes_cell_cycle(adata)
scv.pl.scatter(adata, color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95],
               save='CellCycle-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-CCPhase.png')
s_genes, g2m_genes = scv.utils.get_phase_marker_genes(adata)
#s_genes = scv.get_df(adata[:, s_genes], 'spearmans_score', sort_values=True).index
#g2m_genes = scv.get_df(adata[:, g2m_genes], 'spearmans_score', sort_values=True).index
#kwargs = dict(frameon=False, ylabel='cell cycle genes')
#scv.pl.scatter(adata, list(s_genes[:2]) + list(g2m_genes[:3]), **kwargs)
#save='CellCycle-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.png'

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['Sample type'],
                                 min_mass=3.4, 
                                 save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-SampleType.svg')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['celltype3_pub'],
                                 min_mass=3.4, legend_loc='right',
                                 save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-celltype3_pub.svg')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['Sample type'],
                                 min_mass=3.25, 
                                 save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-SampleType.pdf')

In [ ]:
sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:

scv.pl.velocity_embedding_stream(adata, basis='umap',color=['celltype3_merged'], 
                                 legend_loc='best', 
                                save='-velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn-celltype3.png') #min_mass=3.25,

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden'], legend_loc='on data')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color=['leiden_TNK'], legend_loc='on data')

In [ ]:
sc.set_figure_params()

In [ ]:
sc.pl.umap(adata,color=['XCL1','CCR7','SELL','MKI67', 'STMN1','ENTPD1','CD38','TOX','CD4','CD8A','CD8B', 'CD3D'], color_map='viridis')

In [ ]:
goi=['CD68','CD163','TREM1','MARCO','G0S2','CCL4','CXCL2','AREG','IL1B','TREM2','SPP1','MSR1','CX3CR1','MAF',
                        'SIGLEC1', 'CD38','ENTPD1','CD9','C3','FCGR3A','FCGR1A','C1QA','CSF3R','FCN1','VCAN','S100A8',
     'S100A12','ISG15','CXCL9','CXCL10','CXCL11','CXCL16', 'CD14','EGR1','LYVE1','FN1', 'INHBA','NLRP3']
sc.pl.umap(adata,color=goi, color_map='viridis')

In [ ]:
sc.pl.dotplot(adata, var_names=goi, groupby='leiden', dendrogram=True)

In [ ]:
goi=['CD68','CD163','TREM1','MARCO','G0S2','CCL4','CXCL2','AREG','IL1B','TREM2','SPP1','MSR1','CX3CR1','MAF',
                        'SIGLEC1', 'CD38','ENTPD1','CD9','C3','FCGR3A','FCGR1A','C1QA','CSF3R','FCN1','VCAN','S100A8',
     'S100A12','ISG15','CXCL9','CXCL10','CXCL11','CXCL16', 'CD14','EGR1','LYVE1','FN1', 'INHBA','NLRP3']

sc.pl.dotplot(subdata, var_names=goi, groupby='leiden', dendrogram=True)

In [ ]:
sc.pl.dotplot(adata, var_names=goi, 
              groupby='celltype3_pub', dendrogram=True)

In [ ]:
sc.pl.dotplot(subdata, var_names=goi, 
              groupby='celltype3_pub', dendrogram=True)

In [ ]:
scv.pl.velocity_embedding(adata, dpi=120, arrow_size=2, arrow_length=2,color=['leiden','celltype3_merged'])

In [ ]:
sc.pl.umap(adata,color=['leiden'],legend_loc='on data')

In [ ]:
sc.pl.umap(adata,color=['leidenf'],legend_loc='on data')

In [ ]:
sc.pl.umap(subdata,color=['leiden'],legend_loc='on data')

In [ ]:
subdata=subdata[subdata.obs.sample(frac=0.5).index].copy()

In [ ]:
cr.settings.verbosity = 2
cr.tl.terminal_states(subdata, cluster_key='leiden', weight_connectivities=0.2)


In [ ]:
cr.pl.terminal_states(subdata)

cr.tl.initial_states(subdata, cluster_key='leiden') 


In [ ]:
cr.pl.initial_states(subdata, discrete=True)


In [ ]:
cr.pl.terminal_states(subdata, discrete=True)


In [ ]:
cr.tl.lineages(subdata)


In [ ]:
cr.pl.lineages(subdata, same_plot=False)


In [ ]:
import matplotlib.pyplot as plt
plt.hist(subdata.obs['initial_states_probs'])


In [ ]:

plt.hist(subdata.obs['terminal_states_probs'])


In [ ]:
scv.tl.recover_latent_time(subdata, root_key='initial_states_probs', end_key='terminal_states_probs')


In [ ]:



scv.tl.paga(subdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


In [ ]:
sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:
sc.set_figure_params()

In [ ]:
cr.pl.cluster_fates(subdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


In [ ]:
cnames=['5','7','12','13','14','16','19','22','23','29']
tconf=scv.get_df(subdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden.tsv',sep='\t')

tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
#tran_conf

In [ ]:
#sns.clustermap(tran_conf)

In [ ]:
cr.tl.lineage_drivers(subdata, use_raw=False, backward=False)

In [ ]:
sc.pl.umap(subdata,color=list(subdata.var['to 14'].sort_values().index[1:13]))

In [ ]:
sc.pl.umap(subdata,color=list(subdata.var['to 14'].sort_values(ascending=False).index[1:13]))

In [ ]:
subdata.var['to 14'].sort_values(ascending=False).to_csv(figdir+'/CellRank_TopLineageGenes_'+velosubset+'-'+mysubset+'_'+subcat+'_subset.tsv',sep='\t')

In [ ]:
list(subdata.var['to 14'].sort_values(ascending=False).index[0:20])

In [ ]:
list(subdata.var['to 14'].sort_values(ascending=True).index[0:20])

In [ ]:
cr.pl.lineage_drivers(subdata, lineage="14", n_genes=10)

In [ ]:
root_idx = np.where(subdata.obs['initial_states'] == '5')[0][1]
subdata.uns['iroot'] = root_idx
sc.tl.dpt(subdata)

scv.pl.scatter(subdata, color=['leiden', root_idx, 'latent_time', 'dpt_pseudotime'], fontsize=16,
               cmap='viridis', perc=[2, 98], colorbar=True, rescale_color=[0, 1],
               title=['leiden', 'root cell', 'latent time', 'dpt pseudotime'])


In [ ]:
#subcat='TILonly'
#subdata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.subsampled.origmap.h5ad')
subdata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.subsampled.origmap.h5ad')

In [ ]:
cr.pl.initial_states(subdata, discrete=True)


In [ ]:
cr.pl.terminal_states(subdata, discrete=True)

In [ ]:
scv.tl.recover_latent_time(subdata, root_key='initial_states_probs', end_key='terminal_states_probs')


In [ ]:
root_idx = np.where(subdata.obs['initial_states'] == '5')[0][1]
subdata.uns['iroot'] = root_idx
sc.tl.dpt(subdata)

scv.pl.scatter(subdata, color=['leiden', root_idx, 'latent_time', 'dpt_pseudotime'], fontsize=16,
               cmap='viridis', perc=[2, 98], colorbar=True, rescale_color=[0, 1],
               title=['leiden', 'root cell', 'latent time', 'dpt pseudotime'])

In [ ]:
cdata=subdata.copy()

In [ ]:
scv.tl.rank_dynamical_genes(cdata, groupby='leiden')


In [ ]:
df = scv.get_df(cdata, 'rank_dynamical_genes/names')
df.head(10)


In [ ]:
df.head(50).to_csv(figdir+'/TopDynamicGenesPerCluster_'+velosubset+'-'+mysubset+'_'+subcat+'_loom-dyn_Top50Ranked.subsampled.tsv',sep='\t')


In [ ]:
transitiongenes1=cdata.var['to 14'][list(df.head(50)['29'])][np.abs(cdata.var['to 14'][list(df.head(50)['29'])])>0.1].sort_values()
transitiongenes2=cdata.var['to 14'][list(df.head(50)['5'])][np.abs(cdata.var['to 14'][list(df.head(50)['5'])])>0.1].sort_values()
transitiongenes=list(set(transitiongenes1.index).union(set(transitiongenes2.index)))

tamcxgenes1=cdata.var['to 14'][list(df.head(50)['19'])][np.abs(cdata.var['to 14'][list(df.head(50)['19'])])>0.1].sort_values()
tamcxgenes2=cdata.var['to 14'][list(df.head(50)['22'])][np.abs(cdata.var['to 14'][list(df.head(50)['22'])])>0.1].sort_values()
tamcxgenes=list(set(tamcxgenes1.index).union(set(tamcxgenes2.index)))

In [ ]:
cdata.var['to 14'][transitiongenes].sort_values()

In [ ]:
sc.pl.umap(cdata,color=list(cdata.var['to 14'][transitiongenes].sort_values().index))

In [ ]:
cdata.var['to 14'][list(set(tamcxgenes)-set(transitiongenes))].sort_values()

In [ ]:
sc.pl.umap(cdata,color=list(cdata.var['to 14'][list(set(tamcxgenes)-set(transitiongenes))].sort_values().index))

In [ ]:
scv.pl.velocity(cdata, ['VCAN', 'S100A8','SOD2', 'PLAUR', 'HLA-DPB1','MS4A6A'], ncols=2, add_outline=True)


In [ ]:
ofinterest=set([item for sublist in df.head(50).values.tolist() for item in sublist])
ofinterest=cdata.var['to 14'][list(ofinterest)][np.abs(cdata.var['to 14'][list(ofinterest)])>0.1]

In [ ]:
ofinterest

In [ ]:
velogenes=cdata.var.loc[(cdata.var['velocity_genes']==True)&(cdata.var['fit_likelihood']>.1),:]

In [ ]:
len(set(ofinterest.index)-set(velogenes.index))/len(set(ofinterest)) ## How many are specific 

In [ ]:
ofinterest=ofinterest[list(set(ofinterest.index).intersection(set(velogenes.index)))].sort_values()

In [ ]:
top_genes = cdata.var['fit_likelihood'].sort_values(ascending=False).index#[:300]
top_genes1=cdata.var['fit_likelihood'].sort_values(ascending=False).index[:250]

invelo=[]
for x in list(top_genes):
    invelo=invelo+list(set([x]).intersection(set(velogenes.index)))


In [ ]:
scv.pl.heatmap(cdata, var_names=list(ofinterest.index), sortby='latent_time', 
               col_color='leiden', n_convolve=30,show=True,
               font_scale=0.8,  figsize=(6, 6))

In [ ]:
scv.pl.heatmap(cdata, var_names=list(ofinterest.index), sortby='latent_time', 
               col_color='celltype3_pub', n_convolve=30,show=True,
               font_scale=0.8,  figsize=(6, 6))

In [ ]:
sc.pl.umap(cdata,color=list(ofinterest.index))

In [ ]:
rdata=subdata[subdata.obs['RCat'].isin(['R'])].copy()
tfdata=subdata[subdata.obs['RCat'].isin(['TF'])].copy()

nrdata=subdata[subdata.obs['RCat'].isin(['NR_adj'])].copy()
nrnadjdata=subdata[subdata.obs['RCat'].isin(['NR_nadj'])].copy()


In [ ]:
rbdata=subdata[subdata.obs['RCat'].isin(['R','TF'])].copy()
nrbdata=subdata[subdata.obs['RCat'].isin(['NR_adj','NR_nadj'])].copy()

rbdata_til=rbdata[rbdata.obs['Sample type'].isin(['TIL'])].copy()
nrbdata_til=nrbdata[nrbdata.obs['Sample type'].isin(['TIL'])].copy()


In [ ]:
len(rdata)


In [ ]:
len(tfdata)


In [ ]:
len(nrdata)


In [ ]:
len(nrnadjdata)


In [ ]:
sc.set_figure_params(10)

In [ ]:
cr.pl.initial_states(rbdata, discrete=True)


In [ ]:
cr.pl.terminal_states(rbdata, discrete=True)


In [ ]:
cr.pl.initial_states(nrbdata, discrete=True)


In [ ]:
cr.pl.terminal_states(nrbdata, discrete=True)


In [ ]:
cr.pl.initial_states(rdata, discrete=True)


In [ ]:
cr.pl.terminal_states(rdata, discrete=True)


In [ ]:
cr.pl.initial_states(tfdata, discrete=True)


In [ ]:
cr.pl.terminal_states(tfdata, discrete=True)


In [ ]:
cr.pl.initial_states(nrdata, discrete=True)


In [ ]:
cr.pl.terminal_states(nrdata, discrete=True)


In [ ]:
cr.pl.initial_states(nrnadjdata, discrete=True)

In [ ]:
cr.pl.terminal_states(nrnadjdata, discrete=True)


In [ ]:
cr.pl.initial_states(nrbdata_til, discrete=True)


In [ ]:
cr.pl.terminal_states(nrbdata_til, discrete=True)


In [ ]:
cr.pl.initial_states(rbdata_til, discrete=True)


In [ ]:
cr.pl.terminal_states(rbdata_til, discrete=True)


In [ ]:
#nrbdata=subdata[subdata.obs['RCat'].isin(['NR_adj','NR_nadj'])].copy()
#rbdata=subdata[subdata.obs['RCat'].isin(['R','TF'])].copy()


In [ ]:
scv.tl.paga(rbdata_til, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(rbdata_til, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA', 
                   save='-subsettrajectorymap-'+velosubset+'.rbdata_til.PAGA.svg')


In [ ]:
cr.tl.lineage_drivers(rbdata_til)

lin_drivers_sub=rbdata_til.raw.var['to 14']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]

tran_conf_rb_til=pd.DataFrame(rbdata_til.uns['paga']['transitions_confidence'].todense()).transpose()

In [ ]:
cnames=['5','7','12','13','14','16','19','22','23','29']
tconf=scv.get_df(rbdata_til, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
#tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden.rbdata_TIL.tsv',sep='\t')

tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
scv.tl.paga(nrbdata_til, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(nrbdata_til, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                    save='-subsettrajectorymap-'+velosubset+'.nrbdata_til.PAGA.svg')


cr.tl.lineage_drivers(nrbdata_til)

lin_drivers_sub=nrbdata_til.raw.var['to 14']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]

tran_conf_rb_til=pd.DataFrame(nrbdata_til.uns['paga']['transitions_confidence'].todense()).transpose()

In [ ]:
cnames=['5','7','12','13','14','16','19','22','23','29']
tconf=scv.get_df(nrbdata_til, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
#tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden.nrbdata_TIL.tsv',sep='\t')

tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
scv.tl.paga(rbdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(rbdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                    save='-subsettrajectorymap-'+velosubset+'.rbdata.PAGA.svg')


In [ ]:


cr.tl.lineage_drivers(rbdata)

lin_drivers_sub=rbdata.raw.var['to 14']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]

tran_conf_rb=pd.DataFrame(rbdata.uns['paga']['transitions_confidence'].todense()).transpose()

In [ ]:
cnames=['5','7','12','13','14','16','19','22','23','29']
tconf=scv.get_df(rbdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
#tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden.rbdata.tsv',sep='\t')

tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
scv.tl.paga(nrbdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(nrbdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                    save='-subsettrajectorymap-'+velosubset+'.nrbdata.PAGA.svg')


cr.tl.lineage_drivers(nrbdata)

lin_drivers_sub=nrbdata.raw.var['to 14']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]

tran_conf_nrb=pd.DataFrame(nrbdata.uns['paga']['transitions_confidence'].todense()).transpose()

In [ ]:
cnames=['5','7','12','13','14','16','19','22','23','29']
tconf=scv.get_df(nrbdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
#tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden.nrbdata.tsv',sep='\t')

tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
scv.tl.paga(rdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)



cr.pl.cluster_fates(rdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                    save='-subsettrajectorymap-'+velosubset+'.rdata.PAGA.svg')


cr.tl.lineage_drivers(rdata)

lin_drivers_sub=rdata.raw.var['to 14']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]
tran_conf_r=pd.DataFrame(nrbdata.uns['paga']['transitions_confidence'].todense()).transpose()

In [ ]:
cnames=['5','7','12','13','14','16','19','22','23','29']
tconf=scv.get_df(rbdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
#tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden.rdata.tsv',sep='\t')

tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
scv.tl.paga(nrdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


cr.pl.cluster_fates(nrdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                    save='-subsettrajectorymap-'+velosubset+'.nrdata.PAGA.svg')


cr.tl.lineage_drivers(nrdata)

lin_drivers_sub=nrdata.raw.var['to 14']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]
tran_conf_nr=pd.DataFrame(nrdata.uns['paga']['transitions_confidence'].todense()).transpose()

In [ ]:
cnames=['5','7','12','13','14','16','19','22','23','29']
tconf=scv.get_df(nrbdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
#tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden.nrdata.tsv',sep='\t')

tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
scv.tl.paga(tfdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


cr.pl.cluster_fates(tfdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                    save='-subsettrajectorymap-'+velosubset+'.tfdata.PAGA.svg')


cr.tl.lineage_drivers(tfdata)


lin_drivers_sub=tfdata.raw.var['to 14']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]
tran_conf_tf=pd.DataFrame(tfdata.uns['paga']['transitions_confidence'].todense()).transpose()

In [ ]:
cnames=['5','7','12','13','14','16','19','22','23','29']
tconf=scv.get_df(tfdata, 'paga/transitions_confidence', precision=2).T
tconf.index=cnames
tconf.columns=cnames
#tconf.to_csv(figdir+'/Transitionstable_'+velosubset+'-'+mysubset+'_'+subcat+'_leiden.tfdata.tsv',sep='\t')

tconf.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
scv.tl.paga(nrnadjdata, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')


cr.pl.cluster_fates(nrnadjdata, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA',
                    save='-subsettrajectorymap-'+velosubset+'.nrnadjdata.PAGA.svg')


cr.tl.lineage_drivers(nrnadjdata)

lin_drivers_sub=nrnadjdata.raw.var['to 14']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]
tran_conf_nrnadj=pd.DataFrame(nrnadjdata.uns['paga']['transitions_confidence'].todense()).transpose()

### Genes for specific paths 

In [ ]:
sc.set_figure_params(10)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)

In [ ]:
#subp

In [ ]:
sc.pl.umap(subdata,color=['CD3D','CD2','MSR1','TREM2','CD63'], color_map='viridis')

In [ ]:
sc.pl.dotplot(subdata,var_names=['CD3D','CD2', 'GZMA','MSR1','TREM2','HLA-DRA','CD63' ,
                                 'G0S2','NLRP3', 'CXCL9', 'ISG15','MARCO','IL1B','SPP1'], 
              groupby='leiden', dendrogram=True)

In [ ]:
#subp=rbdata[rbdata.obs['leiden'].isin(['5','29','19'])].copy()
#subp=rbdata[rbdata.obs['leiden'].isin(['5','29','19'])].copy()
#subp=rbdata[rbdata.obs['leiden'].isin(['22','14'])].copy()
#[nrbdata.obs['leiden'].isin(['7','22','5','29','19','14','12','23','16','13'])].copy()

#subp=nrdata[nrdata.obs['leiden'].isin(['7','22','5','29','19','14','12','23','16','13'])].copy()
#subp=subdata[subdata.obs['leiden'].isin(['7','13','16','22','23'])].copy()
#subp=subdata[subdata.obs['leiden'].isin(['7','13','16','22','23'])].copy()
subp=subdata[subdata.obs['leiden'].isin(['16','22','14'])].copy()

#subp=subp[subp.obs['RCat'].isin(['NR_adj','NR_nadj'])].copy()
subp=subp[subp.obs['RCat'].isin(['TF','R'])].copy()

#subp=subp.copy()
scv.tl.paga(subp, groups='leiden', root_key='initial_states_probs', end_key='terminal_states_probs', 
            use_time_prior='velocity_pseudotime')



In [ ]:
cr.pl.cluster_fates(subp, mode="paga_pie", cluster_key="leiden", basis='umap',
                    legend_kwargs={'loc': 'top right out'}, legend_loc='top left out',
                    node_size_scale=1, edge_width_scale=1, max_edge_width=4, title='directed PAGA')


In [ ]:
cr.tl.lineage_drivers(subp)

In [ ]:
lin_drivers_sub=subp.raw.var['to 14']

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]

In [ ]:


lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:15]

lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=True)[0:15]

In [ ]:
set(velogenes.index).intersection(set(lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=True).index))

In [ ]:
up=set(velogenes.index).intersection(set(lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=True)[0:40].index))

In [ ]:
down=set(velogenes.index).intersection(set(lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:40].index))

In [ ]:
##down=set(lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False)[0:40].index)

In [ ]:
up

In [ ]:
down

In [ ]:
lin_drivers_sub[np.abs(lin_drivers_sub)>0.05].sort_values(ascending=False).to_csv(figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_rbdata_leiden_14-16-22.tsv',sep='\t')

In [ ]:
figdir+'/Lingenes_'+velosubset+'-'+mysubset+'_'+subcat+'_rbdata_leiden_5-29-19.tsv'

In [ ]:
scv.pl.velocity(subp, list(up)[0:6], ncols=2, add_outline=True,color='leiden')

In [ ]:
adatapbmc=adata[adata.obs['Sample type']=='PBMC'].copy()
adatatil=adata[adata.obs['Sample type']=='TIL'].copy()

In [ ]:
set(adata.obs['celltype3_pub'])

In [ ]:
adatanocc=adata[adata.obs['celltype3_pub']!='non-classical monocyte'].copy()

In [ ]:
scv.tl.latent_time(adatanocc,min_likelihood=0.2, min_confidence=0.9) # #,root_key='iroot'
#scv.tl.latent_time(adata)
scv.pl.scatter(adatanocc, color='latent_time', color_map='gnuplot', size=30)

In [ ]:
scv.tl.latent_time(adatapbmc,min_likelihood=0.2, min_confidence=0.9) # #,root_key='iroot'
#scv.tl.latent_time(adata)
scv.pl.scatter(adatapbmc, color='latent_time', color_map='gnuplot', size=30)

In [ ]:
scv.tl.latent_time(adatatil,min_likelihood=0.2, min_confidence=0.9) # #,root_key='iroot'
#scv.tl.latent_time(adata)
scv.pl.scatter(adatatil, color='latent_time', color_map='gnuplot', size=30)

In [ ]:
cdata=adata.copy()

In [ ]:
scv.tl.latent_time(cdata,min_likelihood=0.2, min_confidence=0.9) # #,root_key='iroot'
#scv.tl.latent_time(adata)
scv.pl.scatter(cdata, color='latent_time', color_map='gnuplot', size=30)

In [ ]:
#sc.pl.umap(adata[adata.obs['root_cells']>0.5], color=['leiden','celltype3'])

In [ ]:
sc.pl.umap(cdata, color=['root_cells','end_points'])

In [ ]:
sc.pl.umap(adatanocc, color=['root_cells','end_points'])

In [ ]:
#adata.write("/pstore/data/biomics/_pre_portfolio/ONC/scSEQ/ROCHE_INHOUSE/Schlieren_Project_Support_8/Fullanalysis/CD8Tcells-loom-dyn.pathNaitoTExh.h5ad")
#adata.write("/pstore/data/biomics/_pre_portfolio/ONC/scSEQ/ROCHE_INHOUSE/Schlieren_Project_Support_8/Fullanalysis/CD8Tcells-loom-dyn.h5ad")
#adata.write("/pstore/data/biostats/users/kamthont/scvelo/loom.h5ad", compression="gzip")
#adata=scv.read("/pstore/data/biostats/users/kamthont/scvelo/loom.h5ad")
#adata.write("/pstore/data/biomics/_pre_portfolio/ONC/scSEQ/ROCHE_INHOUSE/Schlieren_Project_Support_8/Fullanalysis/CD8Tcells-"+mysubset+"_"+subcat+"_loom-dyn.h5ad")
adata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')



In [ ]:
adata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')

In [ ]:
ddata=cdata.copy()

In [ ]:
ddata.obs['R_clus']=ddata.obs['RCat'].astype(str)+'-'+[x.replace("-", "_") for x in ddata.obs['leiden'].astype(str)]
ddata.obs['Pat_clus']=ddata.obs['PatientID'].astype(str)+'-'+[x.replace("-", "_") for x in ddata.obs['leiden'].astype(str)]
dfPat = ddata.obs.groupby('Pat_clus')['velocity_length', 'velocity_confidence','latent_time'].mean().T

In [ ]:
dfPat

In [ ]:
Resp=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0] for x in list(dfPat.columns)]
PatientID=[x.split('-')[0]  for x in list(dfPat.columns)]
ClusterID=[x.split('-')[1]  for x in list(dfPat.columns)]

In [ ]:
mydf=pd.DataFrame([list(dfPat.loc['velocity_length',:]),Resp,PatientID,ClusterID]).transpose()
mydf.index=dfPat.columns
mydf.columns=['velocity_length','Response','PatientID','Cluster']

mydfl=pd.DataFrame([list(dfPat.loc['latent_time',:]),Resp,PatientID,ClusterID]).transpose()
mydfl.index=dfPat.columns
mydfl.columns=['latent_time','Response','PatientID','Cluster']

In [ ]:
mydf.groupby('PatientID', as_index=False)['velocity_length']

In [ ]:
color_dict = {'R': 'coral', 'TF': 'firebrick', 'NR_nadj': 'lightskyblue','NR_adj': 'royalblue'}

#### Velocity test first

In [ ]:
#corder=['8','6','1','21']
mydf['velocity_length']=mydf['velocity_length'].astype('float')
velMean=mydf.groupby('PatientID')['velocity_length'].mean()
patid=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0]  for x in list(velMean.index)]
totest=pd.DataFrame([list(velMean),patid]).transpose()
totest.columns=['Val','Cat']

In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='R',:]['Val']),list(totest.loc[totest['Cat']=='NR_nadj',:]['Val']))

In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='TF',:]['Val']),list(totest.loc[totest['Cat']=='NR_adj',:]['Val']))

In [ ]:
set(mydf['Response'])

In [ ]:
totestk=totest.copy()

In [ ]:
fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
fig.figure.savefig(figdir+'Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.pdf') 

In [ ]:
totest=list(itertools.combinations(['R','TF','NR_nadj','NR_adj'], 2))
pwilc={}
pt={}
for pairs in totest:
    pwilc[pairs[0]+'-'+pairs[1]]=stats.mannwhitneyu(list(totestk.loc[totestk['Cat']==pairs[0],:]['Val']), 
                           list(totestk.loc[totestk['Cat']==pairs[1],:]['Val']))[1]
    pt[pairs[0]+'-'+pairs[1]]=stats.ttest_ind(list(totestk.loc[totestk['Cat']==pairs[0],:]['Val']), 
                           list(totestk.loc[totestk['Cat']==pairs[1],:]['Val']))[1]

myps=pd.DataFrame([pwilc,pt]).transpose()
myps.columns=['MannWhitney','T-test']

myps.to_csv(figdir+'Pval-Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.pdf',sep='\t')

In [ ]:
myps

In [ ]:
corder=list(set(mydf['Cluster']))

In [ ]:
for i in corder:
    sns.boxplot(y='velocity_length',x='Response',data=mydf.loc[mydf['Cluster']==i,:],
                palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
    sns.swarmplot(x='Response',y='velocity_length',data=mydf.loc[mydf['Cluster']==i,:],
                  color='black',order=['R','TF','NR_nadj','NR_adj'])
    print('Cluster '+i)
    plt.figure()

#### Latent time next

In [ ]:
mydfl

In [ ]:
#corder=['8','6','1','21']
mydfl['latent_time']=mydfl['latent_time'].astype('float')
velMean=mydfl.groupby('PatientID')['latent_time'].mean()
patid=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0]  for x in list(velMean.index)]
totest=pd.DataFrame([list(velMean),patid]).transpose()
totest.columns=['Val','Cat']

ss.mannwhitneyu(list(totest.loc[totest['Cat']=='R',:]['Val']),list(totest.loc[totest['Cat']=='NR_nadj',:]['Val']))



In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='TF',:]['Val']),list(totest.loc[totest['Cat']=='NR_adj',:]['Val']))

In [ ]:
totestk=totest.copy()

In [ ]:
fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
fig.figure.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.pdf') 

In [ ]:
corder=list(set(mydf['Cluster']))

for i in corder:
    sns.boxplot(y='latent_time',x='Response',data=mydfl.loc[mydf['Cluster']==i,:],
                palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
    sns.swarmplot(x='Response',y='latent_time',data=mydfl.loc[mydfl['Cluster']==i,:],
                  color='black',order=['R','TF','NR_nadj','NR_adj'])
    print('Cluster '+i)
    plt.figure()

### Latent time

The dynamical model recovers the latent time of the underlying cellular processes. This latent time represents the cell’s internal clock and approximates the real time experienced by cells as they differentiate, based only on its transcriptional dynamics. 

In [ ]:
scv.tl.rank_dynamical_genes(cdata, groupby='leiden')


In [ ]:
df = scv.get_df(cdata, 'rank_dynamical_genes/names')
df.head(10)
#r

In [ ]:
scv.tl.rank_dynamical_genes(cdata, groupby='celltype3_pub')


In [ ]:
df = scv.get_df(cdata, 'rank_dynamical_genes/names')
df.head(10)
#r

In [ ]:
scv.tl.rank_dynamical_genes(cdata, groupby='leiden')


In [ ]:
df = scv.get_df(cdata, 'rank_dynamical_genes/names')
df.head(10)
#r

In [ ]:
df.head(50).to_csv(figdir+'/TopDynamicGenesPerCluster_'+velosubset+'-'+mysubset+'_'+subcat+'_loom-dyn_Top50Ranked.tsv',sep='\t')

In [ ]:
ofinterest=set([item for sublist in df.head(25).values.tolist() for item in sublist])



In [ ]:
scv.pl.velocity(cdata, ['VCAN', 'S100A8','FCN1'], ncols=2, add_outline=True)

In [ ]:
scv.pl.velocity(cdata, ['BCL2A1', 'CD36','FGR'], ncols=2, add_outline=True)

In [ ]:
velogenes=cdata.var.loc[(cdata.var['velocity_genes']==True)&(cdata.var['fit_likelihood']>.1),:]

In [ ]:
velogenes.sort_values('fit_likelihood', ascending=False)

In [ ]:
set(list(velogenes.index))

In [ ]:
top_genes = cdata.var['fit_likelihood'].sort_values(ascending=False).index#[:300]


In [ ]:
top_genes1=cdata.var['fit_likelihood'].sort_values(ascending=False).index[:250]

In [ ]:
invelo=[]
for x in list(top_genes):
    invelo=invelo+list(set([x]).intersection(set(velogenes.index)))

In [ ]:
scv.pl.heatmap(cdata, var_names=top_genes1, sortby='latent_time', 
               col_color='leiden', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
scv.pl.heatmap(cdata, var_names=invelo, sortby='latent_time', 
               col_color='celltype3_pub', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
scv.pl.heatmap(cdata, var_names=invelo[0:50], 
               sortby='latent_time', col_color='celltype3_pub', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
scv.pl.heatmap(cdata, var_names=invelo[50:100], 
               sortby='latent_time', col_color='celltype3_pub', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
if (len(invelo)>100):
    scv.pl.heatmap(cdata, var_names=invelo[100:len(invelo)], 
               sortby='latent_time', col_color='celltype3_pub', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
velogenes.sort_values('fit_likelihood', ascending=False).to_csv(figdir+'/VelocityInfoPerGene_'+velosubset+'-'+mysubset+'_'+subcat+'_loom-dyn_allVeloGenesRanked.tsv',sep='\t')

In [ ]:
tkey='latent_time'
var_names=invelo
var_names = [name for name in var_names if name in adata.var_names]
xkey='Ms'
time = cdata.obs[tkey].values
time = time[np.isfinite(time)]

df = pd.DataFrame(cdata[:, var_names].layers[xkey][np.argsort(time)], columns=var_names)

In [ ]:
list(adata.obs.columns)

In [ ]:
max_sort = np.argsort(np.argmax(df.values, axis=0))
df = pd.DataFrame(df.values[:, max_sort], columns=df.columns[max_sort])
df.index=pd.DataFrame([cdata.obs['celltype3_merged'][np.argsort(time)],
                       cdata.obs['leiden'][np.argsort(time)],cdata.obs['latent_time'][np.argsort(time)]]).transpose().index
df=pd.concat([df,pd.DataFrame([cdata.obs['celltype3_merged'][np.argsort(time)],
                               cdata.obs['leiden'][np.argsort(time)],cdata.obs['latent_time'][np.argsort(time)]]).transpose()], axis=1)

In [ ]:
def getMeans(adata,myvar):
    gene_ids = adata.raw.var.index.values
    clusters = adata.obs[myvar].cat.categories
    obs = adata.raw[:,gene_ids].X.toarray()
    obs = pd.DataFrame(obs,columns=gene_ids,index=adata.obs[myvar])
    average_obs = obs.groupby(level=0).mean()
    obs_bool = obs.astype(bool)
    fraction_obs = obs_bool.groupby(level=0).sum()/obs_bool.groupby(level=0).count()
    return(average_obs,fraction_obs)


In [ ]:
avg,fct=getMeans(cdata,'leiden')

In [ ]:
avg.loc[:,ofinterest]

In [ ]:
scv.settings.set_figure_params('scvelo')

plt.figure(figsize=(4,3))
mydat=ddata.obs.loc[:,['RCat','latent_time']]
fig=sns.violinplot(x='RCat',y='latent_time',data=mydat, palette=color_dict, order=['R','TF','NR_nadj','NR_adj'])
fig.figure.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response-violins.pdf') 

In [ ]:
lt={}
for resp in  list(color_dict.keys()):
    lt[resp]=cdata[cdata.obs['RCat']==resp].obs['latent_time']
mydat=cdata.obs.loc[:,['RCat','latent_time']]

plt.figure(figsize=(4,3))
myc=list(color_dict.values())
i=0
# Iterate through the five airlines
for resp in list(color_dict.keys()):
    subset = lt[resp]
    sns.distplot(subset, hist = False, kde = True,
                 kde_kws = {'linewidth': 3},
                 label = resp,color=myc[i])
    i=i+1

# Plot formatting
plt.legend(prop={'size': 10}, title = 'Responders')
plt.title('Density Plot across Responses')
plt.xlabel('Latent time')
plt.ylabel('Density')
#plt.savefig(outpath+'/LatentTime_RvsNR_CD8TNK_CCtoExh_TILonly.pdf')
plt.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response-desity.pdf')

In [ ]:
### check avg. per patient as well
r_nr=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='R','latent_time'], y=mydat.loc[mydat['RCat']=='NR_nadj','latent_time'])[1]
### check avg. per patient as well
r_tf=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='R','latent_time'], y=mydat.loc[mydat['RCat']=='TF','latent_time'])[1]
### check avg. per patient as well
r_pd=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='R','latent_time'], y=mydat.loc[mydat['RCat']=='NR_adj','latent_time'])[1]
### check avg. per patient as well
tf_pd=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='TF','latent_time'], y=mydat.loc[mydat['RCat']=='NR_adj','latent_time'])[1]
### check avg. per patient as well
tf_nr=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='TF','latent_time'], y=mydat.loc[mydat['RCat']=='NR_nadj','latent_time'])[1]
### check avg. per patient as well
pd_nr=ss.mannwhitneyu(x=mydat.loc[mydat['RCat']=='NR_adj','latent_time'], y=mydat.loc[mydat['RCat']=='NR_nadj','latent_time'])[1]
myp=pd.Series([r_nr,r_tf,r_pd,tf_pd,tf_nr,pd_nr])
myp.index=['R-NR_nadj','R-TF','R-NR_adj','TF-NR_adj','TF-NR_nadj','NR_adj-NR_nadj']
myp

In [ ]:
plt.figure(figsize=(4,3))
(-1*np.log10((myp+1.163760e-80).sort_values())).plot.bar()
plt.ylabel('-log10(p)')
plt.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response-violins-pvals.pdf') 

### Top-likelihood genes

Driver genes display pronounced dynamic behavior and are systematically detected via their characterization by high likelihoods in the dynamic model. 

In [ ]:
#top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index
scv.pl.scatter(cdata, basis=list(velogenes.sort_values('fit_likelihood', ascending=False).index)[0:10], ncols=5, frameon=False)

In [ ]:
scv.pl.velocity(cdata, var_names=['VCAN','CD36','ZNF331',  'HLA-DQA1', 'CD83','SOD2','FGR'], ncols=2)

In [ ]:
scv.pl.velocity(cdata, var_names=['LST1','FCN1'], ncols=2)

### Cluster-specific top-likelihood genes

Moreover, partial gene likelihoods can be computed for a each cluster of cells to enable cluster-specific identification of potential drivers.

In [ ]:
scv.tl.rank_dynamical_genes(cdata, groupby='leiden')
df = scv.get_df(cdata, 'rank_dynamical_genes/names')
df.head(5)

In [ ]:
df.shape

In [ ]:
test_genes=df.head(5).drop_duplicates

In [ ]:
! jupyter nbconvert --to html scvelo-MonoMacro.ipynb